In [1]:
%reset -fs

In [2]:
import pandas as pd
import numpy as np
import time, os
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
from pymongo import MongoClient
import pandas as pd
import datetime

In [3]:
client = MongoClient()

In [4]:
db = client.nba_db
cursor = db.betting_data.find()
mongo_df = pd.DataFrame(list(cursor))

In [5]:
today = datetime.datetime.today()
max_date = datetime.datetime.strptime(max(mongo_df.Date), '%Y-%m-%d')

def get_date_list(date):
    date_list = []
    today = datetime.datetime.today()
    max_date = datetime.datetime.strptime(max(mongo_df.Date), '%Y-%m-%d')
    delta = today - max_date
    date_list = [max_date - datetime.timedelta(days=x) for x in range(0, delta.days)]
    date_list = [d.strftime('%Y-%m-%d') for d in date_list]
    date_list.append((datetime.datetime.today() - datetime.timedelta(days=1)).strftime('%Y-%m-%d'))
    date_list.append(datetime.datetime.today().strftime('%Y-%m-%d'))
    date_list = set(date_list)
    date_list = list(date_list)
    return date_list
    
    
    

In [6]:
date_list = get_date_list(today)


In [7]:
date_list

['2022-04-01', '2022-03-31']

In [8]:
chromedriver = "/Applications/chromedriver"
os.environ['webdriver.chrome.driver'] = chromedriver
driver = webdriver.Chrome(chromedriver)

In [9]:
bet_search =  'https://www.bettingpros.com/nba/picks/prop-bets/bet/points/?date='

In [10]:
def get_site_rb(date):
    bet_query = bet_search + date
    driver.get(bet_query)
    time.sleep(5)
    try:
        element = driver.find_element_by_xpath('//*[@id="onetrust-accept-btn-handler"]')
        action = ActionChains(driver)
        action.move_to_element(element).click().perform()
    except:
        pass
    get_data_rb()


In [11]:
def get_data_rb():
    data_str = []
    data = []
    li = []
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    table = soup.find('table')
    date = soup.find('h1').text
    date = date.split(" ")[5]
    date.strip(')')
    date = date.replace('(',"").replace(')',"")
    house_accuracy = soup.find('div', class_='best-props__accuracy').text
    table_data = table.tbody.find_all('tr')
    for tr in table_data:
        data_str.append(tr.text.replace('\n', ' ').strip())
    for i in data_str:
        li = list(i.split('            '))
        data.append(li)
    for i in data:
        if len(i) == 7:
            i.append('TBD')
            i.append(date)
            i.append(house_accuracy)
        else:
            i.append(date)
            i.append(house_accuracy)
    global final_df
    final_df = pd.concat([final_df, pd.DataFrame(data)])
    time.sleep(1)

    try:
        page_progress = driver.find_element_by_xpath('//div[contains(@class, "table__pagination")]').text.strip("«").strip("»").strip()
        if int(page_progress[5]) == int(page_progress[10]):
            return final_df
        else:
            element = driver.find_element_by_xpath('/html/body/div[2]/main/div/div/div/div[4]/div/div/a[2]')
            action = ActionChains(driver)
            action.move_to_element(element).click().perform()
            time.sleep(3)
            get_data_rb()
    except:
        return final_df

In [12]:
final_df = pd.DataFrame()

In [13]:
for i in date_list:
    get_site_rb(i)

In [14]:
driver.close()

In [15]:
final_df

,0,1,2,3,4,5,6,7,8,9
0,1 A. Aaron Gordon,DEN - F,vs. MIN,Points,15.50 (-115)7.00Diff: -8.50 Line: 15.50...,15.5,-115 TBD,TBD,04-01-2022,Last 7 days Top 15 Accuracy: 41-61 (40%) Sea...
1,2 T. Terance Mann,LAC - G,@ MIL,Points,16.50 (-127)8.60Diff: -7.90 Line: 16.50...,16.5,-127 TBD,TBD,04-01-2022,Last 7 days Top 15 Accuracy: 41-61 (40%) Sea...
2,3 R. Robert Covington,LAC - F,@ MIL,Points,11.50 (-115)4.70Diff: -6.80 Line: 11.50...,11.5,-115 TBD,TBD,04-01-2022,Last 7 days Top 15 Accuracy: 41-61 (40%) Sea...
3,4 T. Theo Maledon,OKC - G,vs. DET,Points,18.50 (-122)11.70Diff: -6.80 Line: 18.5...,18.5,-122 TBD,TBD,04-01-2022,Last 7 days Top 15 Accuracy: 41-61 (40%) Sea...
4,5 L. Luke Kennard,LAC - G,@ MIL,Points,16.50 (-103)9.90Diff: -6.60 Line: 16.50...,16.5,-103 TBD,TBD,04-01-2022,Last 7 days Top 15 Accuracy: 41-61 (40%) Sea...
...,...,...,...,...,...,...,...,...,...,...
13,64 A. Ayo Dosunmu,CHI - G,vs. LAC,Points,9.50 (+105)9.80Diff: 0.30 Line: 9.50 Co...,9.5,+105,8 (under),03-31-2022,Last 7 days Top 15 Accuracy: 46-55 (46%) Sea...
14,65 M. Marvin Bagley III,DET - F,vs. PHI,Points,15.50 (-107)15.80Diff: 0.30 Line: 15.50...,15.5,-107,8 (under),03-31-2022,Last 7 days Top 15 Accuracy: 46-55 (46%) Sea...
15,66 R. Reggie Jackson,LAC - G,@ CHI,Points,17.50 (EVEN)17.30Diff: -0.20 Line: 17.5...,17.5,EVEN,34 (over),03-31-2022,Last 7 days Top 15 Accuracy: 46-55 (46%) Sea...
16,67 B. Bruce Brown Jr.,BKN - G,vs. MIL,Points,9.50 (EVEN)9.30Diff: -0.20 Line: 9.50 C...,9.5,EVEN,23 (over),03-31-2022,Last 7 days Top 15 Accuracy: 46-55 (46%) Sea...


In [16]:
#Clean up below
final_df.columns=['Name', 'Team', 'Opp', 'Type', 'Line', 'Pick', 'Moneyline', 'Result', 'Date', 'Accuracy']

In [17]:
final_df.reset_index(drop=True, inplace=True)

In [18]:
player_name = final_df.Name.str.split(" ", expand = True)
final_df['Name'] = player_name[2] + ' ' + player_name[4]

In [19]:
team = final_df.Team.str.split(" ", expand = True)
final_df['Team'] = team[0]
final_df['Position'] = team[2]

In [20]:
opp = final_df.Opp.str.split(" ", expand = True)
final_df.Opp = opp[1]
final_df['Home/Away'] = opp[0]

In [21]:
bet_info = final_df.Line.str.split(" ", expand = True)
final_df['Line'] = bet_info[7]
final_df['Projection'] = bet_info[15]
final_df['Diff'] = bet_info[9]
final_df['Moneyline'] = bet_info[13]
final_df['over_under'] = bet_info[16]
final_df['over_under'] = final_df['over_under'].fillna('o')

In [22]:
final_df = final_df[['Date', 'Name', 'Team', 'Position', 'Opp','Home/Away', 'Line', 'Moneyline', 'Projection', 'Diff', 'Pick', 'Result', 'Accuracy', 'over_under']]

In [23]:
final_df = final_df.copy()

In [24]:
final_df['Projection'] = final_df['Projection'].map(lambda x: x.rstrip('ou'))

In [25]:
accuracies = final_df.Accuracy.str.split(" ", expand = True)

In [26]:
final_df['seven_day_accuracy'] = accuracies[8]

In [27]:
final_df.seven_day_accuracy = final_df.seven_day_accuracy.replace('', 50, regex=True)

In [28]:
final_df['seven_day_accuracy'] = final_df['seven_day_accuracy'].map(lambda x: x.lstrip('(').rstrip('%)'))

In [29]:
final_df['seven_day_accuracy'] = pd.to_numeric(final_df['seven_day_accuracy'])

In [30]:
final_df['seven_day_accuracy'] = final_df['seven_day_accuracy'].map(lambda x: x/100)

In [31]:
final_df.drop(columns=['Accuracy'], inplace=True)

In [32]:
final_df['Date'] = pd.to_datetime(final_df['Date'])

In [33]:
# Final form betting df, additional cleaning and saving to csv below
sorted_df = final_df.sort_values(by=['Name', 'Date'])
sorted_df.reset_index(inplace=True, drop=True)

In [34]:
sorted_df['Line'] = pd.to_numeric(sorted_df.Line)

In [35]:
sorted_df = sorted_df.replace('EVEN', '+100')

In [36]:
sorted_df['Moneyline'] = pd.to_numeric(sorted_df.Moneyline)

In [37]:
sorted_df['Result'] = sorted_df.Result.apply(lambda x: str(x))

In [38]:
results = sorted_df.Result.str.split(' ', expand=True);

In [39]:
try:
    sorted_df.Result = results[3]
except:
    sorted_df.Result = results[0]

In [40]:
sorted_df = sorted_df.replace('TBD', None)

In [41]:
sorted_df['Projection'] = pd.to_numeric(sorted_df.Projection)
sorted_df['Diff'] = pd.to_numeric(sorted_df.Diff)
sorted_df['Result'] = pd.to_numeric(sorted_df.Result)
sorted_df['Pick'] = pd.to_numeric(sorted_df.Pick)

In [42]:
# Determine if website pick was a win or loss (for fun, potential spinoff angle).
conditions = [
    ((sorted_df['Result'] < sorted_df['Pick']) & (sorted_df['over_under'] == 'u')),
    ((sorted_df['Result'] < sorted_df['Pick']) & (sorted_df['over_under'] == 'o')),
    ((sorted_df['Result'] > sorted_df['Pick']) & (sorted_df['over_under'] =='o')),
    ((sorted_df['Result'] > sorted_df['Pick']) & (sorted_df['over_under'] == 'u')),
    (sorted_df['Result'].isna() == True)
    ]

# create a list of the values we want to assign for each condition
values = ['1', '0', '1', '0', np.nan]

# create a new column and use np.select to assign values to it using our lists as arguments
sorted_df['successful_pick'] = np.select(conditions, values)

In [43]:
sorted_df

,Date,Name,Team,Position,Opp,Home/Away,Line,Moneyline,Projection,Diff,Pick,Result,over_under,seven_day_accuracy,successful_pick
0,2022-03-31,Bogdan,ATL,G,CLE,vs.,16.5,-115,14.8,-1.7,16.5,11.0,u,0.46,1
1,2022-04-01,Aaron Gordon,DEN,F,MIN,vs.,15.5,-115,7.0,-8.5,15.5,NaN,u,0.40,nan
2,2022-04-01,Aaron Wiggins,OKC,G,DET,vs.,14.5,-106,11.2,-3.3,14.5,NaN,u,0.40,nan
3,2022-04-01,Al Horford,BOS,C,IND,vs.,9.5,-115,12.2,2.7,9.5,NaN,o,0.40,nan
4,2022-04-01,Aleksej Pokusevski,OKC,C,DET,vs.,15.5,-149,14.2,-1.3,15.5,NaN,u,0.40,nan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
148,2022-03-31,Tyrese Maxey,PHI,G,DET,@,16.5,-113,18.8,2.3,16.5,13.0,o,0.46,0
149,2022-03-31,Wesley Matthews,MIL,G,BKN,@,5.5,-115,5.1,-0.4,5.5,11.0,u,0.46,0
150,2022-04-01,Will Barton,DEN,G,MIN,vs.,15.5,-113,16.1,0.6,15.5,NaN,o,0.40,nan
151,2022-04-01,Zach Collins,SAS,F,POR,vs.,7.5,105,6.3,-1.2,7.5,NaN,u,0.40,nan
